In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import (
    Activation,
    Input,
    Dense,
    Flatten,
    Dropout,
    Conv2D,
    MaxPooling2D,
    BatchNormalization,
)
from tensorflow.keras.utils import to_categorical


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/My Drive/Projects/genre_detection

[WinError 3] The system cannot find the path specified: '/content/drive/My Drive/Projects/genre_detection'
d:\programs\TA_projects\model_production\genre_detection


C:\Users\Ricky\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


### Functions ✨

In [ ]:
def plot_history(history):
    fig, axs = plt.subplots(2)

    # accuracy subplot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy level")

    # error subplot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

    plt.figure(figsize=(15,7))

    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

def build_model(input_shape):
    model = Sequential(
        [
            Conv2D(
                16,
                (3, 3),
                strides=(1, 1),
                activation="relu",
                input_shape=input_shape,
                padding="same",
            ),
            MaxPooling2D((2, 2), strides=(2, 2)),
            BatchNormalization(),
            Dropout(0.4),
            Conv2D(32, (3, 3), strides=(1, 1), padding="same", activation="relu"),
            MaxPooling2D((2, 2), strides=(2, 2), padding="same"),
            BatchNormalization(),
            Dropout(0.4),
            Conv2D(64, (3, 3), strides=(1, 1), padding="same", activation="relu",),
            MaxPooling2D((2, 2), strides=(2, 2), padding="same"),
            BatchNormalization(),
            Dropout(0.4),
            Conv2D(128, (3, 3), strides=(1, 1), padding="same", activation="relu"),
            MaxPooling2D((2, 2), strides=(2, 2), padding="same"),
            BatchNormalization(),
            Dropout(0.4),
            Conv2D(256, (3, 3), strides=(1, 1), padding="same", activation="relu"),
            MaxPooling2D((2, 2), strides=(2, 2), padding="same"),
            BatchNormalization(),
            Dropout(0.4),
            Conv2D(512, (3, 3), strides=(1, 1), padding="same", activation="relu"),
            MaxPooling2D((2, 2), strides=(2, 2), padding="same"),
            BatchNormalization(),
            Dropout(0.4),
            Flatten(),
            Dropout(0.5),
            Dense(
                128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.002)
            ),
            Dropout(0.3),
            Dense(
                10, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.002)
            ),
        ]
    )
    return model


def predict(model, X, y):
    X = X[np.newaxis, ...]
    print('X ==>', X.shape)
    prediction = model.predict(X)
    predicted_index = np.argmax(prediction, axis=1)
    print('prediction ==> ',prediction)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))

### Load Dataset 🧪

In [ ]:
classes = ['blues', 'classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

data = np.load('X_train.npy')
labels = np.load('y_train.npy')

labels = to_categorical(labels, num_classes=len(classes))

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

Data successfully loaded!
X shape ==> (9986, 259, 13)
X_test ==> (2497, 259, 13)
(2497, 259, 13, 1)
(5991, 259, 13, 1)
(259, 13, 1)


In [ ]:
input_shape =X_train[0].shape
print(input_shape)

(259, 13, 1)


### Create Model 🔨

In [ ]:
model = build_model(input_shape)

optimizer = keras.optimizers.Adam(learning_rate=0.0005)


model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

model.summary()

### Compile Model 🚀

In [ ]:
reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.97,
    patience=3,
    verbose=1,
    mode="min",
    min_delta=0.0001,
    cooldown=2,
    min_lr=1e-10,
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_validation, y_validation),
    batch_size=16,
    epochs=300,
    callbacks=[reduceLROnPlat],
)

### Evaluate Model 📐  

In [ ]:
plot_history(history)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print("\nTest Accuracy:", test_acc)

In [ ]:
from tensorflow.keras.models import load_model
X_to_predict = X_test[731]
y_to_predict = y_test[731]

# model = load_model('cnn__genre_detection.h5')
# print(X_test.shape)
# print(X_to_predict)
# print(X_to_predict.shape)
predict(model, X_to_predict, y_to_predict)

X ==> (1, 150, 150, 1)
1/1 [==============================] - 0s 28ms/step
prediction ==>  [[1.2724632e-06 3.7919943e-07 9.9405003e-01 2.2347156e-07 4.3064734e-08
  2.4481847e-06 2.6876128e-06 4.2405711e-07 2.5324507e-05 5.9172669e-03]]
Target: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.], Predicted label: [2]


In [ ]:
def majority_vote(scores):
    values, counts = np.unique(scores, return_counts=True)
    ind = np.argmax(counts)
    return values[ind]


preds = model.predict(X_test, batch_size=128, verbose=0)
# print(preds.shape)
# Each sound was divided into 39 segments in our custom function
scores_songs = np.split(np.argmax(preds, axis=1), 2497)
# print(scores_songs)

scores_songs = [majority_vote(scores) for scores in scores_songs]
# Same analysis for split
# print(y_test.shape)
label = np.split(y_test, 2497)
label = [majority_vote(l) for l in label]
from sklearn.metrics import accuracy_score

print(
    "majority voting system (acc) = {:.3f}".format(accuracy_score(label, scores_songs))
)

### Save Model 💾

In [ ]:
model.save('cnn__genre_detection_41100hz_0.95.h5')